In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from  selenium import webdriver
from  selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time
from datetime import datetime
import gzip
from io import BytesIO
import pandas as pd
import xml.etree.ElementTree as ET

In [5]:
def market_price_web_to_df(file,reshet_name):
    url_basic = 'https://url.publishedprices.co.il/file/d/'


    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_experimental_option('detach',True)
    driver = webdriver.Chrome(options=chrome_options)
    driver.get('https://url.publishedprices.co.il/login')
    driver.find_element(By.NAME, value='username').send_keys(reshet_name)
    driver.find_element(By.ID, value='login-button').click()
    
    # time.sleep(10)

    
    file_url = url_basic+file
    
    date_string = file.split('-')[2].replace('.gz','')
    file_date = datetime.strptime(date_string, "%Y%m%d%H%M")
    num_reshet = file.split('-')[0].replace('Price','')
    num_snif = file.split('-')[1]


    cookies = driver.get_cookies()
    session = requests.Session()
    for cookie in cookies:
        session.cookies.set(cookie['name'], cookie['value'])

    response = session.get(file_url)
    try:
        with gzip.open(BytesIO(response.content), 'rb') as gz_file:
            extracted_content = gz_file.read()

        xml_content = extracted_content.decode('utf-8')

        root = ET.fromstring(xml_content)
        data = []
        for item in root.findall('.//Items/Item'):
            row = {child.tag: child.text for child in item}
            data.append(row)
            df = pd.DataFrame(data)
        df['file_name'] = file
        df['num_reshet'] = num_reshet
        df['num_snif'] = num_snif
        df['file_date'] = file_date
    except Exception as e:
        print(e)
        print(file_url)

    driver.quit()
    return df


In [7]:
df = market_price_web_to_df('PriceFull7290876100000-005-202410260010.gz','freshmarket')

In [9]:
def market_promo_web_to_df(file,reshet_name):
    url_basic = 'https://url.publishedprices.co.il/file/d/'


    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_experimental_option('detach',True)
    driver = webdriver.Chrome(options=chrome_options)
    driver.get('https://url.publishedprices.co.il/login')
    driver.find_element(By.NAME, value='username').send_keys(reshet_name)
    driver.find_element(By.ID, value='login-button').click()
    
    # time.sleep(10)

    
    file_url = url_basic+file
    
    date_string = file.split('-')[2].replace('.gz','')
    file_date = datetime.strptime(date_string, "%Y%m%d%H%M")
    num_reshet = file.split('-')[0].replace('Promo','')
    num_snif = file.split('-')[1]


    cookies = driver.get_cookies()
    session = requests.Session()
    for cookie in cookies:
        session.cookies.set(cookie['name'], cookie['value'])

    response = session.get(file_url)
    try:
        with gzip.open(BytesIO(response.content), 'rb') as gz_file:
            extracted_content = gz_file.read()

        xml_content = extracted_content.decode('utf-8')

        root = ET.fromstring(xml_content)
        data = []
        for item in root.findall('.//Promotions/Promotion'):
            row = {child.tag: child.text for child in item}
            data.append(row)
            df = pd.DataFrame(data)
        df['file_name'] = file
        df['num_reshet'] = num_reshet
        df['num_snif'] = num_snif
        df['file_date'] = file_date
    except Exception as e:
        print(e)
        print(file_url)

    driver.quit()
    return df

In [13]:

df_promo = market_promo_web_to_df('PromoFull7290803800003-060-202410261225.gz','yohananof')
df_promo

,PromotionId,PromotionDescription,PromotionUpdateDate,PromotionStartDate,PromotionStartHour,PromotionEndDate,PromotionEndHour,RewardType,AllowMultipleDiscounts,IsWeightedPromo,...,Clubs,MaxQty,MinPurchaseAmnt,Remarks,DiscountRate,DiscountType,file_name,num_reshet,num_snif,file_date
0,154360,שיבולת שועל דק/עבה,2024-05-02 00:00,2016-10-10,00:00:00,2024-12-31,23:59:00,1,1,0,...,\n,NaN,NaN,NaN,NaN,NaN,PromoFull7290803800003-060-202410261225.gz,Full7290803800003,060,2024-10-26 12:25:00
1,165892,שוקיים מקוצר 2 קג ב-60 רשתי,2024-08-26 00:00,2017-11-15,00:00:00,2025-01-04,23:59:00,1,1,1,...,\n,NaN,NaN,NaN,NaN,NaN,PromoFull7290803800003-060-202410261225.gz,Full7290803800003,060,2024-10-26 12:25:00
2,178510,שישליק הודו 2קג ב-65 רשתי,2024-08-26 00:00,2018-12-12,00:00:00,2025-01-04,23:59:00,1,1,1,...,\n,NaN,NaN,NaN,NaN,NaN,PromoFull7290803800003-060-202410261225.gz,Full7290803800003,060,2024-10-26 12:25:00
3,198978,סינטה עגל מעובד מופשר 129.90 רשתי,2024-08-25 00:00,2020-12-01,00:00:00,2025-01-04,23:59:00,1,1,1,...,\n,NaN,NaN,NaN,NaN,NaN,PromoFull7290803800003-060-202410261225.gz,Full7290803800003,060,2024-10-26 12:25:00
4,1107223,שריר הזרוע מעובד מופשר 2ק ב110 רשתי,2024-08-26 00:00,2021-06-06,00:00:00,2025-01-04,23:59:00,1,1,1,...,\n,NaN,NaN,NaN,NaN,NaN,PromoFull7290803800003-060-202410261225.gz,Full7290803800003,060,2024-10-26 12:25:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1340,1323719,קופסאות אחסון אוקסו 1+2 מתנה,2024-09-30 00:00,2024-09-30,00:00:00,2024-11-02,23:59:00,9,1,0,...,\n,NaN,NaN,NaN,10000,0,PromoFull7290803800003-060-202410261225.gz,Full7290803800003,060,2024-10-26 12:25:00
1341,1323922,עוגה בטעם דבש 350 גרם 1+1 מתנה,2024-10-06 00:00,2024-10-06,00:00:00,2024-10-26,23:59:00,9,1,0,...,\n,NaN,NaN,NaN,10000,0,PromoFull7290803800003-060-202410261225.gz,Full7290803800003,060,2024-10-26 12:25:00
1342,1323925,עוגת דבש קפולסקי 1+1 מתנה,2024-10-07 00:00,2024-10-06,00:00:00,2024-10-26,23:59:00,9,1,0,...,\n,NaN,NaN,NaN,10000,0,PromoFull7290803800003-060-202410261225.gz,Full7290803800003,060,2024-10-26 12:25:00
1343,1323985,חטיף נייטיב לחתול 15 גרם 1+2 מתנה,2024-10-07 00:00,2024-10-07,00:00:00,2024-11-30,23:59:00,9,1,0,...,\n,NaN,NaN,NaN,10000,0,PromoFull7290803800003-060-202410261225.gz,Full7290803800003,060,2024-10-26 12:25:00


In [17]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
p = 'נוטרילון'
df_promo[df_promo['PromotionDescription'].str.contains(p, case=False, na=False)]

,PromotionId,PromotionDescription,PromotionUpdateDate,PromotionStartDate,PromotionStartHour,PromotionEndDate,PromotionEndHour,RewardType,AllowMultipleDiscounts,IsWeightedPromo,AdditionalRestrictions,MinQty,DiscountedPrice,DiscountedPricePerMida,MinNoOfItemOfered,WeightUnit,PromotionItems,Clubs,MaxQty,MinPurchaseAmnt,Remarks,DiscountRate,DiscountType,file_name,num_reshet,num_snif,file_date
457,1320894,נוטרילון 800 גרם עד 4 יח מעל 100,2024-08-21 00:00,2024-08-21,00:00:00,2024-10-31,23:59:00,1,1,0,\n,2.00,110.00,55.00,10,1,\n,\n,4.00,100.00,\n,NaN,NaN,PromoFull7290803800003-060-202410261225.gz,Full7290803800003,060,2024-10-26 12:25:00


In [ ]:
#משימות
#הפרדה בין מבצעים לרגיל
#תחקור הפער בין מלא למהלך היום
#שילוב של אינדקס חנויות
#בחירת תזמון ריצה ואיזה קבצים ניקח
#פילטור של NULL

'020'